# Weekly Project 5 
## Implementation of global registration 
### Task 1
Today, your task is to implement a global registration algorithm.

It should be able to roughly align two point clouds.
Implement the global registration, and then try the following:

1. Can you fit `r1.pcd` and `r2.pcd`?
2. Can you fit `car1.ply` and `car2.ply`?
The corresponding files are in the `global_registration` folder.

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# implement Fast Point Feature Histograms without using compute_fpfh_feature
def compute_fpfh_feature_custom(pcd, radius_normal=0.05, radius_feature=0.1):
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    kdtree = o3d.geometry.KDTreeFlann(pcd)

    n_points = len(points)
    fpfh_features = np.zeros((n_points, 33))

    for i in range(n_points):
        [_, idx, _] = kdtree.search_radius_vector_3d(pcd.points[i], radius_feature)
        if len(idx) < 5:
            continue
        u = normals[i]
        v = np.cross(u, np.array([1, 0, 0]))
        if np.linalg.norm(v) < 1e-6:
            v = np.cross(u, np.array([0, 1, 0]))
        v /= np.linalg.norm(v)
        w = np.cross(u, v)

        hist = np.zeros(33)
        for j in idx:
            if j == i:
                continue
            diff = points[j] - points[i]
            diff /= np.linalg.norm(diff)
            alpha = np.dot(v, normals[j])
            phi = np.dot(u, diff)
            theta = np.arctan2(np.dot(w, normals[j]), np.dot(u, normals[j]))

            alpha_bin = int((alpha + 1) / 2 * 11)
            phi_bin = int((phi + 1) / 2 * 11)
            theta_bin = int((theta + np.pi) / (2 * np.pi) * 11)

            alpha_bin = min(max(alpha_bin, 0), 10)
            phi_bin = min(max(phi_bin, 0), 10)
            theta_bin = min(max(theta_bin, 0), 10)

            hist[alpha_bin] += 1
            hist[11 + phi_bin] += 1
            hist[22 + theta_bin] += 1

        hist /= np.linalg.norm(hist) + 1e-6
        fpfh_features[i] = hist

    fpfh_o3d = o3d.pipelines.registration.Feature()
    fpfh_o3d.data = fpfh_features.T
    return fpfh_o3d


### Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in `car_challange` folder.

You can use the notebooks from Monday as a starting point.